# Hyperparameters optimization with Optuna
Date : 10/08/2020

Study the hyperparameters and their best values

In [1]:
import optuna
import neptune
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMGRkNDk3ZWEtZjYyOC00ODYwLWFhZTQtZTY0YmI5YzA5NWZjIn0="
import neptunecontrib.monitoring.optuna as opt_utils
import plotly

In [2]:
# useful imports
from IPython.display import Audio, display
import pandas as pd
import numpy as np

# voicemap imports
from config import PATH, DATA_PATH, LOG_PATH
from voicemap.train import *
from voicemap.datasets import datasets
from voicemap.datasets import standardization

In [3]:
t = TrainingArgs()
t.lr = 0.01
t.filters = 64
t.batch_size = 1500 
t.epochs = 5
t.train_datasets_letters = "LT"
t.val_datasets_letters = "LT"
t.precompute_spect = False
t.spectrogram = True
t.use_standardized = False
print(vars(t))

{'model': 'resnet', 'model_path': None, 'dim': 1, 'lr': 0.01, 'weight_decay': 0.01, 'momentum': 0.9, 'epochs': 5, 'filters': 64, 'batch_size': 32, 'n_seconds': 3, 'downsampling': 4, 'use_standardized': False, 'spectrogram': True, 'precompute_spect': False, 'window_length': 0.02, 'window_hop': 0.01, 'device': 'cuda', 'n_t': 0, 'T': None, 'n_f': 0, 'F': None, 'train_datasets_letters': 'LT', 'val_datasets_letters': 'LT'}


In [4]:
def objective(trial: optuna.trial.Trial):
    if t.use_standardized:
        n_samples_min = trial.suggest_int("n_samples_per_speaker", 30, 100)
        standardization.standardize_speakers("train", t.train_datasets_letters, n_samples_min)
    
    # Suggest ranges over which the parameters will vary
#     t.lr = trial.suggest_categorical("lr", [0.1, 0.01])
#     t.filters = trial.suggest_categorical("filters", [64, 128])
#     t.batch_size = trial.suggest_categorical("batch_size", [128, 32, 1])
#     t.epochs = trial.suggest_int("epochs", 1, 5)

    t.weight_decay = trial.suggest_loguniform("weight_decay", 0.005, 0.95)
    t.momentum = trial.suggest_uniform("momentum", 0, 1)
    #t.spectrogram = trial.suggest_int("spectrogram", 0, 1)
    t.window_length = trial.suggest_loguniform("window_length", 0.01, 0.5)
    t.window_hop = t.window_length / 2.0
#     t.window_hop = trial.suggest_loguniform("window_hop", 0.01, 0.1)
#     if t.window_hop > t.window_length:
#         t.window_hop = trial.suggest_loguniform("window_hop", 0.01, 0.1)
    # t.n_seconds = trial.suggest_uniform("n_seconds", 2.5, 3) # must refactor spectrograms generation before pushing n_seconds above 3
#     t.train_datasets_letters = trial.suggest_categorical("train_datasets_letters", ["LCT", "CT"])
    print(vars(t))
    epochs_results = train(args=t, train_datasets_letters=t.train_datasets_letters, val_datasets_letters=t.val_datasets_letters, optunaTrial=trial, verbose=True)
    print(epochs_results)
    result = epochs_results[-1]['val_loss']
    return result  

In [ ]:
# Setup neptune
neptune.init('team-pep/speaker-identification', api_token=NEPTUNE_API_TOKEN)
neptune_callback = opt_utils.NeptuneCallback(log_charts=True, log_study=True)
neptune.create_experiment(name="weight_decay & momentum", params=t.__dict__)
# Setup optuna
study = optuna.create_study(study_name='study-04', storage=f"sqlite:///{LOG_PATH}/optimization.db")

# Optuna optimization
study.optimize(objective, n_trials=5, callbacks=[neptune_callback])
# Log into neptune
neptune.log_metric("best_val_loss", study.best_value)
opt_utils.log_study_info(study)

https://ui.neptune.ai/team-pep/speaker-identification/e/SID-49


[I 2020-08-24 10:20:32,665] A new study created with name: study-04


{'model': 'resnet', 'model_path': None, 'dim': 1, 'lr': 0.01, 'weight_decay': 0.5586319695705432, 'momentum': 0.3204032283803132, 'epochs': 5, 'filters': 64, 'batch_size': 32, 'n_seconds': 3, 'downsampling': 4, 'use_standardized': False, 'spectrogram': True, 'precompute_spect': False, 'window_length': 0.027384632738151973, 'window_hop': 0.013692316369075986, 'device': 'cuda', 'n_t': 0, 'T': None, 'n_f': 0, 'F': None, 'train_datasets_letters': 'LT', 'val_datasets_letters': 'LT'}
LibriSpeech train-clean-100 loaded from '/home/profenpoche/voicemap/data/train-clean-100.index.csv'
LibriSpeech train-clean-360 loaded from '/home/profenpoche/voicemap/data/train-clean-360.index.csv'
Finished indexing data. 129652 usable files found.
In LibriSpeech ['train-clean-100', 'train-clean-360'] there are  1172  speakers
TCOF loaded from '/home/profenpoche/voicemap/data/TCOF/TCOF_train.csv'
In TCOF  ['train']  there are  652  speakers
LibriSpeech dev-clean loaded from '/home/profenpoche/voicemap/data/dev

Epoch 1: 100%|█████████▉| 4308/4309 [20:03<00:00,  6.35it/s, loss=20.5, accuracy=0]



Epoch 00002: val_loss improved from inf to 15790477.72258, saving model to /home/profenpoche/voicemap/models/model=resnet__dim=1__lr=0.01__weight_decay=0.559__momentum=0.320__filters=64__batch_size=32__n_seconds=3.000__use_standardized=False__spectrogram=True__n_t=0__T=None__n_f=0__F=None__datasets=LT__num_samples=153196__num_classes=1824.pt


Epoch 2: 100%|█████████▉| 4308/4309 [20:15<00:00,  7.75it/s, loss=21, accuracy=0]  



Epoch 00003: val_loss improved from 15790477.72258 to 1007307.97487, saving model to /home/profenpoche/voicemap/models/model=resnet__dim=1__lr=0.01__weight_decay=0.559__momentum=0.320__filters=64__batch_size=32__n_seconds=3.000__use_standardized=False__spectrogram=True__n_t=0__T=None__n_f=0__F=None__datasets=LT__num_samples=153196__num_classes=1824.pt


Epoch 3:  62%|██████▏   | 2651/4309 [12:30<07:33,  3.65it/s, loss=19.4, accuracy=0]

In [ ]:
study = optuna.load_study(study_name='study-01', storage=f"sqlite:///{LOG_PATH}/optimization.db")
study.get_trials()

In [ ]:


# log some metrics

# for i in range(100):
#     neptune.log_metric('loss', 0.95**i)

# neptune.log_metric('AUC', 0.96)